# DRUG DISCOVERY USING QUANTUM COMPUTING

# Step 0 : Import Packages

In [1]:
from utility.MoleculeParser import MoleculeData
from utility.QMUQUBO import QMUQUBO
from utility.AnnealerOptimizer import Annealer
from utility.ResultProcess import ResultParser
import time
timestamp = time.strftime("%Y%m%d-%H")

2022-11-12 14:53:20,622 dwave.cloud INFO MainThread Log level for 'dwave.cloud' namespace set to 0


# Step 1: Prepare Molecular Data

In [2]:
s3_bucket = f"amazon-braket-qceddstack-416584524924-us-west-2" 
prefix = "Qresults"
raw_path = './molecule-data/STI_ideal.mol2'

In [3]:
mol_data = MoleculeData(raw_path, 'qmu')
data_path = mol_data.save("latest")
num_rotation_bond = mol_data.bond_graph.rb_num
print(f"You have loaded the raw molecule data and saved as {data_path}. \n\
This molecule has {num_rotation_bond} rotable bond")

INFO:root:parse mol2 file!
INFO:root:finish save qmu_STI_ideal_data_latest.pickle


You have loaded the raw molecule data and saved as ./qmu_STI_ideal_data_latest.pickle. 
This molecule has 16 rotable bond


# Step 2: Build Model

| Parameter | Description | Value |
|--- |--- |--- |
|M | number of torsions for molecular unfolding| [1, max number of rotatable bonds] |
|D| angle precision of rotation| 8|

In [4]:
init_param = {}
method = ['pre-calc']

for mt in method:
    if mt == 'pre-calc':
        init_param[mt] = {}
        init_param[mt]['param'] = ['M', 'D', 'A', 'hubo_qubo_val']
    
qmu_qubo = QMUQUBO(mol_data, method, **init_param)

INFO:root:initial pre-calculate for constructing molecule QUBO


In [5]:
model_param = {}
num_rotation_bond = mol_data.bond_graph.rb_num
method = 'pre-calc'
model_param[method] = {}
model_param[method]['M'] = [2]
model_param[method]['D'] = [8]
model_param[method]['A'] = [300]
model_param[method]['hubo_qubo_val'] = [200]
qmu_qubo.build_model(**model_param)

INFO:root:Construct model for M:2,D:8,A:300,hubo_qubo_val:200 0.0012605865796407065 min


0

In [6]:
# describe the model parameters
model_info = qmu_qubo.describe_model()

INFO:root:method: pre-calc
INFO:root:The model_name should be {M}_{D}_{A}_{hubo_qubo_val}
INFO:root:param: M, value {2}
INFO:root:param: D, value {8}
INFO:root:param: A, value {300}
INFO:root:param: hubo_qubo_val, value {200}


In [7]:
# save the model
model_path = qmu_qubo.save("latest")

print(f"You have built the QUBO model and saved it as {model_path}")

INFO:root:finish save qmu_STI_ideal_model_latest.pickle


You have built the QUBO model and saved it as ./qmu_STI_ideal_model_latest.pickle


# Step 3: Optimize Configuration

In [8]:
qmu_qubo_optimize = QMUQUBO.load(model_path)

In [9]:
model_info = qmu_qubo_optimize.describe_model()

INFO:root:method: pre-calc
INFO:root:The model_name should be {M}_{D}_{A}_{hubo_qubo_val}
INFO:root:param: M, value {2}
INFO:root:param: D, value {8}
INFO:root:param: A, value {300}
INFO:root:param: hubo_qubo_val, value {200}


In [10]:
# get the model you want to optimize
M = 2
D = 8
A = 300
hubo_qubo_val = 200
model_name = "{}_{}_{}_{}".format(M, D, A, hubo_qubo_val)
method = "pre-calc"

qubo_model = qmu_qubo_optimize.get_model(method, model_name)

# Simulated annealing

In [11]:
method = 'dwave-sa'

optimizer_param = {}
optimizer_param['shots'] = 1000

sa_optimizer = Annealer(qubo_model, method, **optimizer_param)

INFO:root:use simulated annealer from dimod


In [12]:
sa_optimize_result = sa_optimizer.fit()

INFO:root:fit() ...
INFO:root:dwave-sa save to local
INFO:root:finish save dwave-sa_result.pickle


# Quantum annealing

In [13]:
method = 'dwave-qa'

optimizer_param = {}
optimizer_param['shots'] = 1000
optimizer_param['bucket'] = s3_bucket # the name of the bucket
optimizer_param['prefix'] = prefix # the name of the folder in the bucket
optimizer_param['device'] = "arn:aws:braket:::device/qpu/d-wave/DW_2000Q_6"
optimizer_param["embed_method"] = "default"

qa_optimizer = Annealer(qubo_model, method, **optimizer_param)

INFO:root:use quantum annealer arn:aws:braket:::device/qpu/d-wave/DW_2000Q_6 


In [14]:
# not create annealing task, only embedding logic
qa_optimizer.embed()
# create annealing task
qa_optimize_result = qa_optimizer.fit()

INFO:root:fit() ...


FrozenDict({0: {128, 4, 5, 6, 7}, 1: {129, 4, 5, 6, 7}, 2: {130, 4, 5, 6, 7}, 3: {131, 4, 5, 6, 7}, 4: {12}, 8: {136, 12, 13, 14, 15}, 9: {137, 12, 13, 14, 15}, 10: {138, 12, 13, 14, 15}, 11: {139, 12, 13, 14, 15}, 5: {13}, 6: {14}, 7: {15}, 12: {20}, 16: {144, 20, 21, 22, 23}, 17: {145, 20, 21, 22, 23}, 18: {146, 20, 21, 22, 23}, 19: {147, 20, 21, 22, 23}, 13: {21}, 15: {23}, 20: {28}, 24: {152, 28, 29, 30, 31}, 25: {153, 28, 29, 30, 31}, 26: {154, 28, 29, 30, 31}, 27: {155, 28, 29, 30, 31}, 21: {29}, 22: {30}, 23: {31}, 28: {36}, 32: {160, 36, 37, 38, 39}, 33: {161, 36, 37, 38, 39}, 34: {162, 36, 37, 38, 39}, 35: {163, 36, 37, 38, 39}, 29: {37}, 30: {38}, 31: {39}, 36: {44}, 40: {168, 44, 45, 47}, 41: {169, 44, 45, 47}, 42: {170, 44, 45, 47}, 37: {45}, 39: {47}, 44: {52}, 48: {176, 52, 53, 54, 55}, 49: {177, 52, 53, 54, 55}, 50: {178, 52, 53, 54, 55}, 51: {179, 52, 53, 54, 55}, 45: {53}, 47: {55}, 52: {60}, 56: {184, 60, 61, 62, 63}, 57: {185, 60, 61, 62, 63}, 58: {186, 60, 61, 62, 6

INFO:root:finish save /tmp/qa_result.pickle
INFO:root:_upload_result_json, bucket=amazon-braket-qceddstack-416584524924-us-west-2, key=Qresults/a042e012-8595-4b60-a914-554091477ff1/qa_result.pickle
INFO:root:dwave-qa save to s3 - a042e012-8595-4b60-a914-554091477ff1: None


In [15]:
qa_task_id = qa_optimizer.get_task_id()
print(f"task id is {qa_task_id}")

task id is a042e012-8595-4b60-a914-554091477ff1


Finally, we can compare the execution time between SA and QA :

In [16]:
print(f"dwave-sa run time {sa_optimize_result['time']}")
print(f"dwave-qa run time {qa_optimize_result['time']}")

dwave-sa run time 133.82616782188416
dwave-qa run time 6.107104539871216


# Step 4: PostProcess Result

In [17]:
method = "dwave-sa"
sa_param = {}
sa_param["raw_path"] = raw_path
sa_param["data_path"] = data_path

sa_process_result = ResultParser(method, **sa_param)
# print(f"{method} result is {sa_process_result.get_all_result()}")

local_time, _ , _, _= sa_process_result.get_time()

print(f"time for {method}: \n \
    local time is {local_time}")

INFO:root:_load_raw_result
INFO:root:load simulated annealer dwave-sa raw result
INFO:root:MoleculeData.load()
INFO:root:init mol data for final position
INFO:root:init mol data for raw position
INFO:root:_parse_model_info
INFO:root:_init_parameters
INFO:root:parse simulated annealer result
INFO:root:sa only has local_time!


time for dwave-sa: 
     local time is 133.82616782188416


In [18]:
sa_atom_pos_data = sa_process_result.generate_optimize_pts()
# save unfold file for visualization and parameters for experiment: 1. volume value 2. relative improvement
sa_result_filepath, sa_result_json = sa_process_result.save_mol_file(f"{timestamp}")

print(f"result path is {sa_result_filepath}, and result optimization file path is {sa_result_json}")

INFO:root:generate_optimize_pts()
INFO:root:var_dict_raw {'5': ['7'], '6': ['8']} var_dict_list [{'5': '7', '6': '8'}]
INFO:root:chosen var {'x_5_7', 'x_6_8'}
INFO:root:tor list {'X_5_7', 'X_6_8'}
INFO:root:initial 24.294143332232615
INFO:root:optimize 25.166386778061835
INFO:root:optimize_gain 1.0359034452831255
INFO:root:start physical check
INFO:root:save_mol_file 20221112-14
INFO:root:finish save ./molecule-data/STI_ideal_dwave-sa_20221112-14.mol2 and ./molecule-data/STI_ideal_dwave-sa_20221112-14.json


result path is ./molecule-data/STI_ideal_dwave-sa_20221112-14.mol2, and result optimization file path is ./molecule-data/STI_ideal_dwave-sa_20221112-14.json


In [19]:
sa_process_result.parameters

{'volume': {'optimize': 25.166386778061835,
  'initial': 24.294143332232615,
  'gain': 1.0359034452831255,
  'unfolding_results': ['X_5_7', 'X_6_8'],
  'annealing_results': ['x_5_7', 'x_6_8'],
  'optimize_info': {'optimize_state': True, 'result_rank': 5}}}

In [20]:
method = "dwave-qa"
qa_param = {}
qa_param["bucket"] = s3_bucket
qa_param["prefix"] = prefix
qa_param["task_id"] = qa_task_id
qa_param["raw_path"] = raw_path
qa_param["data_path"] = data_path

qa_process_result = ResultParser(method, **qa_param)
# print(f"{method} result is {qa_process_result.get_all_result()}")

local_time, task_time, total_time, access_time = qa_process_result.get_time()

print(f"time for {method}: \n \
    local time is {local_time},\n \
    task time is {task_time}, \n \
    qpu total time is {total_time}, \n \
    qpu access time is {access_time}")

INFO:root:_load_raw_result
INFO:root:load quantum annealer raw result
INFO:root:_read_result_obj
INFO:root:_read_result_obj: Qresults/a042e012-8595-4b60-a914-554091477ff1/qa_result.pickle
INFO:root:MoleculeData.load()
INFO:root:init mol data for final position
INFO:root:init mol data for raw position
INFO:root:_parse_model_info
INFO:root:_init_parameters
INFO:root:parse quantum annealer result
INFO:root:_read_result_obj
INFO:root:_read_result_obj: Qresults/a042e012-8595-4b60-a914-554091477ff1/results.json


time for dwave-qa: 
     local time is 6.107104539871216,
     task time is 3.416, 
     qpu total time is 0.255524, 
     qpu access time is 0.249899


In [21]:
qa_atom_pos_data = qa_process_result.generate_optimize_pts()
# save unfold file for visualization and parameters for experiment: 1. volume value 2. relative improvement
qa_result_filepath, qa_result_json = qa_process_result.save_mol_file(f"{timestamp}")
print(f"result path is {qa_result_filepath}, and result optimization file path is {qa_result_json}")

INFO:root:generate_optimize_pts()
INFO:root:var_dict_raw {'5': ['7'], '6': ['8']} var_dict_list [{'5': '7', '6': '8'}]
INFO:root:chosen var {'x_5_7', 'x_6_8'}
INFO:root:tor list {'X_5_7', 'X_6_8'}
INFO:root:initial 24.294143332232615
INFO:root:optimize 25.166386778061835
INFO:root:optimize_gain 1.0359034452831255
INFO:root:start physical check
INFO:root:save_mol_file 20221112-14
INFO:root:finish save ./molecule-data/STI_ideal_dwave-qa_20221112-14.mol2 and ./molecule-data/STI_ideal_dwave-qa_20221112-14.json


result path is ./molecule-data/STI_ideal_dwave-qa_20221112-14.mol2, and result optimization file path is ./molecule-data/STI_ideal_dwave-qa_20221112-14.json


In [22]:
qa_process_result.parameters

{'volume': {'optimize': 25.166386778061835,
  'initial': 24.294143332232615,
  'gain': 1.0359034452831255,
  'unfolding_results': ['X_5_7', 'X_6_8'],
  'annealing_results': ['x_5_7', 'x_6_8'],
  'optimize_info': {'optimize_state': True, 'result_rank': 1}}}

In [23]:
# this shows the original molecule
qa_process_result.InteractView(raw_path, size=800)

interactive(children=(Text(value='./molecule-data/STI_ideal.mol2', description='mol'), IntSlider(value=800, de…

In [24]:
# copy the variable for the path of unfolding molecule, we can see the unfolding results. this also works for sa_process_result.
qa_process_result.InteractView(qa_result_filepath, size=800)

interactive(children=(Text(value='./molecule-data/STI_ideal_dwave-qa_20221112-14.mol2', description='mol'), In…